In [1]:
import sys
print(sys.executable)

/home/paulamoure/project/rag-exercise-cohere/venv/bin/python


In [2]:
import cohere  # SDK oficial de Cohere para usar sus modelos (embeddings, chat, etc.)
import pandas as pd  # Manipulación de datos tabulares (DataFrames) - ideal para CSVs
import numpy as np  # Operaciones matemáticas y arrays - necesario para embeddings
import os  # Interactuar con el sistema operativo (variables de entorno, archivos)
from typing import List, Dict  # Type hints para mejor código (List[str], Dict[str, Any])
import json  # Leer/escribir archivos JSON - útil para guardar embeddings

# Verificar versiones
print(f"Cohere version: {cohere.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Cohere version: 5.15.0
Pandas version: 2.2.3
Numpy version: 2.2.6


In [ ]:
# Configurar API Key de forma segura
from dotenv import load_dotenv
load_dotenv()

# Cargar API key desde variable de entorno
api_key = os.getenv("COHERE_API_KEY")
if not api_key:
    raise ValueError("COHERE_API_KEY no encontrada en .env")

co = cohere.Client(api_key)
print("Cliente Cohere configurado correctamente")

Cliente Cohere configurado correctamente


In [ ]:
# Probar conexión con Cohere
try:
    # Test simple con un embedding pequeño
    response = co.embed(texts=["Hello world"], model="embed-english-v3.0", input_type="search_document")
    print(f"✅ Conexión exitosa. Embedding dimension: {len(response.embeddings[0])}")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Conexión exitosa. Embedding dimension: 1024


In [8]:
# Crear dataset de documentos de ejemplo
documents = [
    "Cohere provides large language models and NLP tools for businesses.",
    "RAG combines retrieval of relevant documents with text generation.",
    "Vector databases store embeddings for fast similarity search.",
    "Semantic search finds documents based on meaning, not just keywords.",
    "Embeddings are numerical representations of text in high-dimensional space.",
    "Machine learning models can understand context and generate human-like text.",
    "Natural language processing helps computers understand human language.",
    "AI assistants use retrieval-augmented generation for accurate responses."
]

print(f"Dataset creado con {len(documents)} documentos")
for i, doc in enumerate(documents):
    print(f"{i}: {doc}")

Dataset creado con 8 documentos
0: Cohere provides large language models and NLP tools for businesses.
1: RAG combines retrieval of relevant documents with text generation.
2: Vector databases store embeddings for fast similarity search.
3: Semantic search finds documents based on meaning, not just keywords.
4: Embeddings are numerical representations of text in high-dimensional space.
5: Machine learning models can understand context and generate human-like text.
6: Natural language processing helps computers understand human language.
7: AI assistants use retrieval-augmented generation for accurate responses.


In [10]:
# Generar embeddings para los documentos
print("Generando embeddings para los documentos...")

# Crear embeddings para todos los documentos
doc_embeddings = co.embed(
    texts=documents,
    model="embed-english-v3.0",
    input_type="search_document"
)

# Convertir a numpy array para facilitar cálculos
embeddings_array = np.array(doc_embeddings.embeddings)

print(f"Embeddings generados:")
print(f"- Número de documentos: {len(documents)}")
print(f"- Dimensión de embeddings: {embeddings_array.shape[1]}")
print(f"- Shape total: {embeddings_array.shape}")

Generando embeddings para los documentos...
Embeddings generados:
- Número de documentos: 8
- Dimensión de embeddings: 1024
- Shape total: (8, 1024)


In [15]:
# Implementar búsqueda semántica
def semantic_search(query: str, top_k: int = 3) -> List[Dict]:
    """
    Busca los documentos más relevantes para una query usando similitud coseno
    """
    # Generar embedding para la query
    query_embedding = co.embed(
        texts=[query],
        model="embed-english-v3.0",
        input_type="search_query"
    )

    query_vector = np.array(query_embedding.embeddings[0])

    # Calcular similitud coseno con todos los documentos
    similarities = np.dot(embeddings_array, query_vector) / (
        np.linalg.norm(embeddings_array, axis=1) * np.linalg.norm(query_vector)
    )

    # Obtener los top_k más similares
    top_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in top_indices:
        results.append({
            "document": documents[idx],
            "similarity": similarities[idx],
            "index": idx
        })
    return results
print("Función de búsqueda semántica creada")

Función de búsqueda semántica creada


In [16]:
# Probar búsqueda semántica
test_query = "What is vector search?"

print(f"🔍 Buscando: '{test_query}'\n")
results = semantic_search(test_query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"{i}. Similitud: {result['similarity']:.4f}")
    print(f"   Documento: {result['document']}\n")

🔍 Buscando: 'What is vector search?'

1. Similitud: 0.4186
   Documento: Vector databases store embeddings for fast similarity search.

2. Similitud: 0.2750
   Documento: Semantic search finds documents based on meaning, not just keywords.

3. Similitud: 0.2636
   Documento: RAG combines retrieval of relevant documents with text generation.



In [19]:
# Función RAG completa
def rag_response(query: str, top_k: int = 3) -> str:
    """
    Implementa RAG: busca documentos relevantes y genera respuesta
    """
    # 1. RETRIEVAL: Buscar documentos relevantes
    relevant_docs = semantic_search(query, top_k=top_k)

     # 2. Construir contexto para el prompt
    context = "\n".join([doc["document"] for doc in relevant_docs])
    
    # 3. GENERATION: Crear prompt con contexto
    prompt = f"""Based on the following context, answer the question accurately and concisely.

Context:
{context}

Question: {query}

Answer: """

    # 4. Generar respuesta usando Cohere
    response = co.chat(
       message=prompt,
       model="command-r-plus",  # Modelo de chat de Cohere
       temperature=0.3
    )

    return response.text

print("Función RAG completa")


Función RAG completa


In [20]:
# Probar RAG completo
questions = [
    "What is RAG?",
    "How do embeddings work?",
    "What are vector databases used for?"
]

for question in questions:
    print(f"❓ Pregunta: {question}")
    answer = rag_response(question)
    print(f"🤖 Respuesta: {answer}\n")
    print("-" * 50)

❓ Pregunta: What is RAG?
🤖 Respuesta: RAG stands for Retrieval-Augmented Generation.

--------------------------------------------------
❓ Pregunta: How do embeddings work?
🤖 Respuesta: Embeddings are created by converting text into numerical representations, which are then placed in a high-dimensional space.

--------------------------------------------------
❓ Pregunta: What are vector databases used for?
🤖 Respuesta: Vector databases are used for storing embeddings, which are numerical representations of text, to enable fast similarity searches.

--------------------------------------------------
